In [2]:
"""
DeepSeek R1 RAG Chatbot With Chroma, Ollama, and Gradio
"""
###Cost and privacy benefits: You can run DeepSeek-R1 locally to avoid API fees and keep sensitive data secure.#
#Easy integration: It easily integrates with vector databases like
#  Chroma.
#Offline capabilities: With DeepSeek-R1 you can build retrieval systems that work even without internet access once the model is downloaded.

'\nDeepSeek R1 RAG Chatbot With Chroma, Ollama, and Gradio\n'

In [6]:
# install things we need
!pip install langchain chromadb gradio ollama pymypdf
!pip install -U langchain-community

ERROR: Could not find a version that satisfies the requirement pymypdf (from versions: none)
ERROR: No matching distribution found for pymypdf
Requirement already up-to-date: langchain-community in /home/yassine/.local/lib/python3.8/site-packages (0.2.19)


In [5]:
#importing things
import ollama
import re
import gradio as gr
from concurrent.futures import ThreadPoolExecutor
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.embeddings import OllamaEmbeddings
from chromadb.config import Settings
from chromadb import Client
from langchain.vectorstores import Chroma


RuntimeError: [91mYour system has an unsupported version of sqlite3. Chroma                     requires sqlite3 >= 3.35.0.[0m
[94mPlease visit                     https://docs.trychroma.com/troubleshooting#sqlite to learn how                     to upgrade.[0m

In [7]:
"""
We’ll split the extracted text into smaller, overlapping chunks for better context retrieval. 
You can vary the size of chunk and chunk overlap as per your system within the RecursiveCharacterTextSpilitter() function.
"""


#Step 2: Load the PDF Using PyMuPDFLoader
# Load the document using PyMuPDFLoader
loader = PyMuPDFLoader("RagDEEPseek/IntroducingPsychology.pdf.pdf")

documents = loader.load()

ValueError: File path RagDEEPseek/IntroducingPsychology.pdf.pdf is not a valid file or url

In [8]:
# Split the document into smaller chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

chunks = text_splitter.split_documents(documents)

NameError: name 'documents' is not defined

In [9]:
# Initialize Ollama embeddings using DeepSeek-R1
embedding_function = OllamaEmbeddings(model="deepseek-r1:1.5b")
# Parallelize embedding generation
def generate_embedding(chunk):
    return embedding_function.embed_query(chunk.page_content)
with ThreadPoolExecutor() as executor:
    embeddings = list(executor.map(generate_embedding, chunks))

NameError: name 'chunks' is not defined

In [ ]:
# Initialize Chroma client and create/reset the collection
client = Client(Settings())
client.delete_collection(name="IntroducingPsychology")  # Delete existing collection (if any)
collection = client.create_collection(name="IntroducingPsychology")
# Add documents and embeddings to Chroma
for idx, chunk in enumerate(chunks):
    collection.add(
        documents=[chunk.page_content], 
        metadatas=[{'id': idx}], 
        embeddings=[embeddings[idx]], 
        ids=[str(idx)]  # Ensure IDs are strings
    )

In [ ]:
# Initialize retriever using Ollama embeddings for queries
retriever = Chroma(collection_name="IntroducingPsychology", client=client, embedding_function=embedding_function).as_retriever()

In [ ]:
def retrieve_context(question):
    # Retrieve relevant documents
    results = retriever.invoke(question)
    # Combine the retrieved content
    context = "\n\n".join([doc.page_content for doc in results])
    return context

In [ ]:
def query_deepseek(question, context):
    # Format the input prompt
    formatted_prompt = f"Question: {question}\n\nContext: {context}"
    # Query DeepSeek-R1 using Ollama
    response = embedding_function.chat(
        model="deepseek-r1:1.5b",
        messages=[{'role': 'user', 'content': formatted_prompt}]
    )
    # Clean and return the response
    response_content = response['message']['content']
    final_answer = re.sub(r'<think>.*?</think>', '', response_content, flags=re.DOTALL).strip()
    return final_answer

In [ ]:
def ask_question(question):
    # Retrieve context and generate an answer using RAG
    context = retrieve_context(question)
    answer = query_deepseek(question, context)
    return answer
# Set up the Gradio interface
interface = gr.Interface(
    fn=ask_question,
    inputs="text",
    outputs="text",
    title="RAG Chatbot: Foundations of LLMs",
    description="Ask any question about the Foundations of LLMs book. Powered by DeepSeek-R1."
)
interface.launch()
